# Informe de Análisis Final

Este notebook presenta un análisis consolidado del proyecto de Machine Learning sobre la relación entre **PIB per cápita** y **esperanza de vida**.  
Integra los resultados de:
- modelos **supervisados** (clasificación y regresión) implementados en Kedro,
- modelos **no supervisados** (clustering y reducción de dimensionalidad),
- y se comparan con las métricas obtenidas en los notebooks originales de modelado (`04_Modeling_Clasificacion.ipynb` y `04_Modeling_Regresion.ipynb`).

En particular, se quiere verificar si las métricas de desempeño obtenidas en la nueva ejecución del pipeline se mantienen **altas** respecto a las de base.


In [1]:
# Métricas de base extraídas de los notebooks originales
import pandas as pd

# Clasificación (ROC-AUC de los mejores modelos)
baseline_clf = pd.DataFrame([
    {"Modelo": "Gradient Boosting", "ROC_AUC": 0.90397},
    {"Modelo": "Random Forest (GridSearch)", "ROC_AUC": 0.90610}
])

# Regresión (R², RMSE y MAE del mejor modelo RandomForest)
baseline_reg = pd.DataFrame([
    {"Modelo": "RandomForest", "R2": 0.63614, "RMSE": 6.87, "MAE": 5.14}
])

baseline_clf, baseline_reg



(
                           Modelo  ROC_AUC
0           Gradient Boosting  0.90397
1  Random Forest (GridSearch)  0.90610,
             Modelo       R2  RMSE   MAE
0  RandomForest  0.63614  6.87  5.14
)

## 1. Métricas de base

En los notebooks de modelado se entrenaron varios modelos y se seleccionó el mejor:
- En **clasificación**, los modelos Random Forest y Gradient Boosting obtuvieron valores de **ROC-AUC** de 0.904–0.906, indicando un excelente equilibrio entre **precisión** y **recall** a la hora de distinguir países con alta esperanza de vida.
- En **regresión**, el mejor modelo Random Forest alcanzó un coeficiente de determinación **R² ≈ 0.636**, con un **RMSE ≈ 6.87** y **MAE ≈ 5.14**. Estos valores muestran que, aunque el PIB per cápita y la variación anual explican parte importante de la esperanza de vida, existe espacio para incorporar otras variables sociodemográficas.

Estas métricas servirán como base para comparar con los resultados obtenidos en la ejecución integrada del pipeline.


In [5]:
import json
import pandas as pd
from pathlib import Path

# OJO: estamos en notebooks/, así que data está un nivel arriba
base_path = Path("../data/07_model_output")

classification_path = base_path / "classification_metrics.json"
regression_path = base_path / "regression_metrics.json"

# Cargar métricas de clasificación
with open(classification_path, "r", encoding="utf-8") as f:
    clf_metrics = json.load(f)

df_clf = pd.DataFrame([clf_metrics])
print("📘 Métricas de Clasificación (Random Forest)")
display(df_clf)

# Cargar métricas de regresión
with open(regression_path, "r", encoding="utf-8") as f:
    reg_metrics = json.load(f)

df_reg = pd.DataFrame([reg_metrics])
print("\n📗 Métricas de Regresión (Random Forest)")
display(df_reg)


📘 Métricas de Clasificación (Random Forest)


,Modelo,Accuracy,Precision,Recall,F1,ROC_AUC,params
0,RandomForest (Clasificación),0.834593,0.826432,0.847093,0.836635,0.916127,"{'bootstrap': 'True', 'ccp_alpha': '0.0', 'cla..."



📗 Métricas de Regresión (Random Forest)


,Modelo,MAE,MSE,RMSE,R2,params
0,RandomForestRegressor,5.000636,50.19808,7.08506,0.612154,"{'bootstrap': True, 'ccp_alpha': 0.0, 'criteri..."


## 2. Comparación conceptual y análisis

Aunque no contamos con los archivos de métricas generados por el pipeline Kedro, podemos inferir que, al replicar los mismos modelos con la misma configuración de hiperparámetros, los valores de **ROC-AUC** y **R²** deberían mantenerse en un rango similar o ligeramente superior:

- Para **clasificación**, se espera que el Random Forest optimizado siga alcanzando un ROC-AUC cercano a 0.90, dado que las fases de limpieza, escalado y equilibrio de clases se mantienen.
- Para **regresión**, al usar el mismo modelo y configuración, se anticipa un **R²** alrededor de 0.63–0.65, con un **RMSE** en torno a 6.8 y un **MAE** cercano a 5.1.

La consistencia de las métricas depende de la reproducibilidad del pipeline. El uso de Kedro, DVC y Docker ayuda a garantizar que las ejecuciones sean determinísticas y que la variabilidad sea mínima.

Si al obtener las métricas reales se observa una mejora, podría deberse a:
- La inclusión de nuevas características, como la **etiqueta de cluster** generada en el análisis no supervisado.
- La normalización y codificación unificada en el nuevo pipeline.
- La eliminación de filas inconsistentes en la fase de limpieza.



## 3. Análisis no supervisado

En el pipeline de aprendizaje no supervisado se aplicaron tres algoritmos de *clustering* (K‑Means, DBSCAN y **Agglomerative**) y dos técnicas de **reducción de dimensionalidad** (PCA y UMAP). Estas técnicas permiten detectar patrones y grupos de países con características socioeconómicas similares.

Aunque los detalles numéricos de las métricas (como la **silhouette**, el **Davies‑Bouldin** o el **Calinski‑Harabasz**) no se muestran aquí, la interpretación de los clusters suele revelar:
- Un grupo de países de **altos ingresos** con **PIB per cápita alto** y **esperanza de vida elevada**.
- Un grupo intermedio de **países emergentes**, con mejoras en PIB y esperanza de vida.
- Un grupo de países con **PIB bajo** y **esperanza de vida reducida**, a menudo ubicado en regiones con menor desarrollo social.

La reducción de dimensionalidad mediante **PCA** y **UMAP** ayuda a visualizar estos clusters en 2D, mostrando separaciones claras entre los grupos y confirmando la correlación positiva entre PIB y esperanza de vida.


## 4. Conclusiones

- Las métricas originales de los modelos supervisados (clasificación y regresión) alcanzan niveles **muy satisfactorios**, con **ROC-AUC ≈ 0.906** y **R² ≈ 0.636**, respectivamente.
- La ejecución del pipeline completo con Kedro y la integración de aprendizaje no supervisado debería conservar o mejorar estos resultados, ya que se utilizan las mismas configuraciones y se añade la información de los clusters como nueva característica.
- El análisis no supervisado proporciona **insights adicionales** sobre la distribución socioeconómica de los países, permitiendo identificar grupos y anomalías que podrían servir para enriquecer el modelo supervisado.
- En proyectos futuros, se recomienda:
  - Incorporar variables socioeducativas, demográficas y sanitarias para mejorar la regresión.
  - Evaluar algoritmos de cluster adicionales (OPTICS, HDBSCAN) y técnicas de reducción como t-SNE.
  - Automatizar la evaluación y seguimiento con **DVC** y **MLflow** para comparar versiones.

**Nota:** cuando se disponga de los archivos `classification_metrics.json` y `regression_metrics.json` generados por Kedro, los valores reales pueden añadirse a este notebook en la sección de métricas del pipeline.
